In [25]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import numpy as np
from numpy import NaN, NAN,nan
import statistics

all_data={}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    all_data[column]=yf.download(column, start = '2014-06-02', end='2016-06-13')

luvdf=pd.read_csv('LUV.csv')
luvdf['Date'] = pd.to_datetime(luvdf['Date'])
luvdf.index = luvdf['Date']
luvdf = luvdf.drop(columns = ['Date'])
luvdf = luvdf.iloc[21:]

dict1={ 'Date':np.array([k for k in all_data['AAL'].index])}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    dict1[column]=np.array([k for k in all_data[column]['Adj Close']])
price_industry = pd.DataFrame(dict1) 
price_industry.index = price_industry['Date']
price_industry = price_industry.drop(columns = ['Date'])

price_return=price_industry.pct_change(periods = 1)
price_return=price_return.replace(to_replace=[NaN],value=0)

lag=pd.DataFrame({'price_LUV':np.array([k for k in luvdf['Adj Close']]),
                  'daily_return_airline_industry' :price_return.mean(axis=1),
                'MovingAverage(5)':luvdf['Adj Close'].rolling(5).mean(),
                  'MovingAverage(25)':luvdf['Adj Close'].rolling(25).mean()
               },index=[i for i in all_data['AAL'].index])
lag

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25)
2014-06-02,14.711601,0.000000,NaN,NaN
2014-06-03,14.475902,0.000875,NaN,NaN
2014-06-04,14.259844,-0.005360,NaN,NaN
2014-06-05,14.289306,0.002586,NaN,NaN
2014-06-06,14.338411,0.014908,14.415013,NaN
...,...,...,...,...
2016-06-07,2.170000,0.016486,2.128000,2.2488
2016-06-08,2.160000,0.008763,2.128000,2.2464
2016-06-09,2.110000,0.025104,2.136000,2.2372
2016-06-10,2.070000,-0.029168,2.134000,2.2252


In [26]:
dict3={ 'Date':np.array([k for k in luvdf.index]),
       'Volume_LUV':np.array([k for k in luvdf['Volume']])
      }   
volume = pd.DataFrame(dict3) 
volume.index = volume['Date']
volume = volume.drop(columns = ['Date'])

volume_return=volume.pct_change(periods = 1)
volume_return=volume_return.replace(to_replace=[NaN],value=0)
volume_return.mean()
volume_change=np.array(volume_return)

price=np.array(luvdf['Adj Close'])
maximum=0
ma_small=0
ma_big=0

for j in range(10,35,5):
    MA_BIG=np.array(luvdf['Adj Close'].rolling(j).mean())
    for i in range(2,j//2+1):
        MA_SMALL=np.array(luvdf['Adj Close'].rolling(i).mean())
        
        MA_small_industry=np.array(price_industry.mean(axis=1).rolling(i).mean())
        MA_big_industry=np.array(price_industry.mean(axis=1).rolling(j).mean())
        signals_industry=[]
        for m in range(0,MA_small_industry.shape[0]-1):
            if MA_small_industry[m]==nan or MA_big_industry[m]==nan:
                signals_industry.append(0)
                continue
            if MA_small_industry[m]>=MA_big_industry[m] and MA_small_industry[m-1]<MA_big_industry[m-1]:
                signals_industry.append(1)
                continue
            elif MA_small_industry[m]<=MA_big_industry[m] and MA_small_industry[m-1]>MA_big_industry[m-1]:
                signals_industry.append(-1)
                continue
            else:
                signals_industry.append(0)

        signals=[] 
        for k in range(0,MA_BIG.shape[0]-1):
            if MA_SMALL[k]==nan or MA_BIG[k]==nan:
                signals.append(0)
                continue
            elif (MA_SMALL[k]>=MA_BIG[k] and MA_SMALL[k-1]<MA_BIG[k-1]and (volume_change[k+1]-volume_change[k-1])>0.40):
                flag1=False
                for n in range(k-1,k-9,-1):
                    if(signals_industry[n]==1):
                        signals.append(1)
                        flag1=True
                        break
                if(flag1==False):               
                    signals.append(0)

                        
            elif (MA_SMALL[k]<=MA_BIG[k] and MA_SMALL[k-1]>MA_BIG[k-1]and (volume_change[k-1]-volume_change[k+1])>0.40):
                flag2=False
                for n in range(k-1,k-9,-1):
                    if(signals_industry[n]==-1):
                        signals.append(-1)
                        flag2=True
                        break
                if(flag2==False):               
                    signals.append(0)
            else:
                signals.append(0)
        signals.append(0)
        signal=np.array(signals)  
        Pnl=[price[l] if signal[l]== 1 else -price[l] if signal[l] ==-1 else 0 for l in  range(0,len(price)-1)]

            
        print(i,j,sum(Pnl))
        if(sum(Pnl)>maximum):
            optimized_Pnl=Pnl
            optimized_signal=signal
            maximum=sum(Pnl)
            ma_small=i
            ma_big=j
print(optimized_signal,ma_small,ma_big,maximum)
x=[]
for i in range(len(optimized_signal)):
    if optimized_signal[i]!=0:
        x.append(i+1)
print(x)
lag['SIGNAL']=np.array(optimized_signal)
optimized_Pnl.append(sum(optimized_Pnl))
lag['P/L']=np.array(optimized_Pnl)
lag

2 10 6.2452630000000005
3 10 22.057094000000003
4 10 0.13220699999999974
5 10 -0.16473700000000058
2 15 -11.374887000000001
3 15 -5.21
4 15 -20.713504999999998
5 15 -3.37
6 15 -12.059999999999999
7 15 -5.97
2 20 6.280000000000001
3 20 -20.794178
4 20 -2.46
5 20 -1.2000000000000002
6 20 0
7 20 0
8 20 0
9 20 3.2400000000000007
10 20 -16.545651
2 25 5.380000000000001
3 25 -0.6500000000000004
4 25 -9.100000000000001
5 25 -3.6900000000000004
6 25 1.0700000000000003
7 25 -5.8
8 25 -5.5200000000000005
9 25 -13.58
10 25 -23.845651000000004
11 25 -25.345651
12 25 -15.86
2 30 -6.48
3 30 -8.530000000000001
4 30 -2.2
5 30 -2.2
6 30 0
7 30 0
8 30 -4.85
9 30 -18.115651
10 30 -1.98
11 30 -1.98
12 30 -1.98
13 30 -5.4
14 30 -1.98
15 30 -0.33000000000000007
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 

,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25),SIGNAL,P/L
2014-06-02,14.711601,0.000000,NaN,NaN,0,0.000000
2014-06-03,14.475902,0.000875,NaN,NaN,0,0.000000
2014-06-04,14.259844,-0.005360,NaN,NaN,0,0.000000
2014-06-05,14.289306,0.002586,NaN,NaN,0,0.000000
2014-06-06,14.338411,0.014908,14.415013,NaN,0,0.000000
...,...,...,...,...,...,...
2016-06-07,2.170000,0.016486,2.128000,2.2488,0,0.000000
2016-06-08,2.160000,0.008763,2.128000,2.2464,0,0.000000
2016-06-09,2.110000,0.025104,2.136000,2.2372,0,0.000000
2016-06-10,2.070000,-0.029168,2.134000,2.2252,0,0.000000


In [32]:
import datetime
import pandas as pd
import numpy as np

class trading_strategy:


    def __init__(self):

        self.todays_date = datetime.datetime.now().date()
        self.index = pd.date_range(self.todays_date, periods=1)
        self.columns = ['AdjClose']
        self.df= pd.DataFrame(index=self.index, columns=self.columns)
        self.df=self.df.drop(self.df.index[0])
        self.moving_average_10=[]
        self.moving_average_3=[]
        self.pnl=0
    
    def process_tick(self, adujusted_price):
        self.todays_date+=datetime.timedelta(days=1)
        row = pd.Series({'AdjClose':adujusted_price},name=self.todays_date)
        self.df = self.df.append(row)
        self.moving_average_3=np.array(self.df['AdjClose'].rolling(3).mean())
        self.moving_average_10=np.array(self.df['AdjClose'].rolling(10).mean())
        
        signal_received=self.check_signal()
        if(signal_received==1):
            self.generate_buy_order(adujusted_price)
        
        elif(signal_received==-1):
            self.generate_sell_order(adujusted_price)
            
        self.update_pnl(signal_received,adujusted_price)
   
            
    def generate_buy_order(self,adjusted_price):
        print("BUY",adjusted_price,self.todays_date)
    def generate_sell_order(self,adjusted_price):
        print("SELL",adjusted_price,self.todays_date)    
    def check_signal(self):
        current_tick=len(self.moving_average_3)-1
        if self.moving_average_3[current_tick]==nan or self.moving_average_10[current_tick]==nan:
            return 0
        elif self.moving_average_3[current_tick]>=self.moving_average_10[current_tick] and self.moving_average_3[current_tick-1]<self.moving_average_10[current_tick-1]:
            return 1
        elif self.moving_average_3[current_tick]<=self.moving_average_10[current_tick] and self.moving_average_3[current_tick-1]>self.moving_average_10[current_tick-1]:
            return -1
        else:
            return 0     
    def update_pnl(self,signal_received,adujusted_price):
        if(signal_received==1):
            self.pnl+=adujusted_price
        elif(signal_received==-1):
            self.pnl-=adujusted_price
        else:
            self.pnl+=0
    def display_pnl(self):
        print("pnl=",self.pnl)

        
ts_new=trading_strategy()
list1=luvdf['Adj Close'].values
for i in list1:
    ts_new.process_tick(i)

ts_new.display_pnl()


BUY 14.279485000000001 2020-02-26
SELL 15.448164000000002 2020-03-15
BUY 14.780348000000002 2020-04-08
SELL 13.179553 2020-04-12
BUY 13.849301 2020-04-18
SELL 13.849301 2020-04-26
BUY 9.457094 2020-05-28
SELL 9.180108 2020-05-31
BUY 8.873445 2020-06-05
SELL 8.992153 2020-06-06
BUY 8.804198 2020-06-12
SELL 9.19 2020-06-24
BUY 6.73 2020-07-09
SELL 7.43 2020-07-18
BUY 6.25 2020-08-08
SELL 6.25 2020-08-20
BUY 6.35 2020-08-30
SELL 5.65 2020-09-02
BUY 5.43 2020-09-13
SELL 5.11 2020-09-16
BUY 5.22 2020-09-17
SELL 6.36 2020-10-02
BUY 6.21 2020-10-08
SELL 5.73 2020-10-12
BUY 5.27 2020-10-28
SELL 5.28 2020-11-01
BUY 5.37 2020-11-03
SELL 5.36 2020-11-07
BUY 5.39 2020-11-08
SELL 5.48 2020-11-18
BUY 3.57 2020-12-31
SELL 3.24 2021-01-07
BUY 3.76 2021-01-12
SELL 3.25 2021-01-18
BUY 3.39 2021-01-26
SELL 3.47 2021-02-01
BUY 3.41 2021-02-03
SELL 3.38 2021-02-05
BUY 3.42 2021-02-06
SELL 3.1 2021-02-09
BUY 3.45 2021-02-14
SELL 3.33 2021-02-22
BUY 3.62 2021-03-01
SELL 3.42 2021-03-08
BUY 2.37 2021-03-23
SE

In [33]:
#cross verifying result
MA_5=np.array(luvdf['Adj Close'].rolling(3).mean())
MA_25=np.array(luvdf['Adj Close'].rolling(10).mean())
signals=[]
for i in range(0,MA_5.shape[0]):
    if MA_5[i]==nan or MA_25[i]==nan:
        signals.append(0)
        continue
    if MA_5[i]>=MA_25[i] and MA_5[i-1]<MA_25[i-1]:
        signals.append(1)
        continue
    elif MA_5[i]<=MA_25[i] and MA_5[i-1]>MA_25[i-1]:
        signals.append(-1)
        continue
    else:
        signals.append(0)
signals.append(0)
signal=np.array(signals)  
Pnl=[price[l] if signal[l]== 1 else -price[l] if signal[l] ==-1 else 0 for l in  range(0,len(price))]
print(sum(Pnl))

0.9745919999999964


In [35]:
luvdf=yf.download("LUV", start = '2017-06-02', end='2019-06-13')
ts_new2=trading_strategy()
list1=luvdf['Adj Close'].values
for i in list1:
    ts_new2.process_tick(i)

ts_new2.display_pnl()

[*********************100%***********************]  1 of 1 completed
BUY 58.58316421508789 2020-02-23
SELL 60.32249450683594 2020-03-11
BUY 51.49345016479492 2020-04-21
SELL 55.671756744384766 2020-05-24
BUY 53.830970764160156 2020-06-09
SELL 63.875274658203125 2020-07-08
BUY 63.894798278808594 2020-07-18
SELL 60.713233947753906 2020-07-24
BUY 56.78019332885742 2020-08-09
SELL 56.614280700683594 2020-08-18
BUY 56.97048568725586 2020-08-21
SELL 57.087852478027344 2020-08-23
BUY 58.388641357421875 2020-08-24
SELL 56.51081466674805 2020-09-01
BUY 54.593868255615234 2020-09-20
SELL 52.69648361206055 2020-09-24
BUY 51.845584869384766 2020-10-10
SELL 49.733036041259766 2020-10-14
BUY 50.35898208618164 2020-10-18
SELL 49.95798873901367 2020-10-20
BUY 50.378108978271484 2020-10-29
SELL 50.20152282714844 2020-11-07
BUY 50.87845993041992 2020-11-13
SELL 50.927513122558594 2020-11-26
BUY 55.62685775756836 2020-11-28
SELL 60.640602111816406 2021-01-09
BUY 57.01101303100586 2021-01-26
SELL 56.92248